# Task for Today  

***

## Predicting COVID-19 Mortality Based on Diet  

Given *data about various countries and their daily intakes of different food groups*, let's try to predict whether a given country will have a **high or low COVID-19 mortality rate**.

We will use a TensorFlow ANN to make our predictions.

# Getting Started

In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import tensorflow as tf

In [ ]:
data = pd.read_csv('../input/covid19-healthy-diet-dataset/Food_Supply_Quantity_kg_Data.csv')

In [ ]:
data

In [ ]:
data.info()

# Preprocessing

In [ ]:
data = data.drop('Unit (all except Population)', axis=1)

## Missing Values

In [ ]:
data.isna().sum()

In [ ]:
for column in data.columns:
    if data.dtypes[column] != 'object' and data.isna().sum()[column] > 0:
        data[column] = data[column].fillna(data[column].mean())

### Dealing with the Undernourished column

In [ ]:
data['Undernourished'].value_counts()

In [ ]:
undernourished_numeric = data.loc[data['Undernourished'] != '<2.5', 'Undernourished'].astype(np.float)
undernourished_numeric

In [ ]:
undernourished_numeric = undernourished_numeric.fillna(undernourished_numeric.mean())
undernourished_numeric = pd.qcut(undernourished_numeric, q=3, labels=[1, 2, 3])
undernourished_numeric

In [ ]:
data.loc[undernourished_numeric.index, 'Undernourished'] = undernourished_numeric

In [ ]:
data['Undernourished'] = data['Undernourished'].apply(lambda x: 0 if x == '<2.5' else x)

In [ ]:
data['Undernourished'].value_counts()

In [ ]:
data

## Feature and Target Selection

In [ ]:
data = data.drop('Country', axis=1)

data = data.drop(['Confirmed', 'Recovered', 'Active'], axis=1)

In [ ]:
pd.qcut(data['Deaths'], q=2, labels=[0, 1]).value_counts()

In [ ]:
data['Deaths'] = pd.qcut(data['Deaths'], q=2, labels=[0, 1])

## Splitting and Scaling

In [ ]:
data

In [ ]:
y = data['Deaths']
X = data.drop('Deaths', axis=1)

In [ ]:
scaler = StandardScaler()

X = scaler.fit_transform(X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8)

# Training

In [ ]:
X.shape

In [ ]:
inputs = tf.keras.Input(shape=(26,))
x = tf.keras.layers.Dense(64, activation='relu')(inputs)
x = tf.keras.layers.Dense(64, activation='relu')(x)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)


model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=[
        'accuracy',
        tf.keras.metrics.AUC(name='auc')
    ]
)


batch_size = 64
epochs = 14

history = model.fit(
    X_train,
    y_train,
    validation_split=0.2,
    batch_size=batch_size,
    epochs=epochs,
    verbose=0
)

# Results

In [ ]:
fig = px.line(
    history.history,
    y=['loss', 'val_loss'],
    labels={'index': "Epoch", 'value': "Loss"},
    title="Training and Validation Loss"
)

fig.show()

In [ ]:
np.argmin(history.history['val_loss'])

In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
len(y_test)

# Data Every Day  

This notebook is featured on Data Every Day, a YouTube series where I train models on a new dataset each day.  

***

Check it out!  
https://youtu.be/LLC8rmmVOjM